## Vacuum agent with Wider perception (5 Stanze)

Modifiche:
5 stanze invece che 3
L'agente va verso la stanza sporca più vicina

Le modifiche sono indicate dai commenti che iniziano con "Edit"

In [1]:
from tkinter import *
from PIL import Image, ImageTk
import random
import sys
import os.path
import math
from agents import *

loc_A, loc_B, loc_C, loc_D, loc_E = (0, 0), (1, 0), (2, 0), (3, 0), (4, 0)  # The three locations for the Vacuum world

MyImage = None

class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's status.
    Extension: its also perceive the closer dirty room if exists """

    def __init__(self, root, height=400, width=700):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean',
                       loc_D: 'Clean',
                       loc_E: 'Clean'
                      }
        self.closer_dirty_room=-1
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()
        

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Agent has now \n \t the perception ', self.status, ' \n \t being at Location=', agent.location, 
              ' \n \t with performance ', agent.performance)
        return (agent.location, self.status)

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        (Col, Row) = agent.location
        if action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                self.buttons[agent.location[0]].config(bg='white', activebackground='light grey')
                agent.performance += 10
            self.status[agent.location] = 'Clean'
        elif action == 'Right' :
            if agent.location[0] < 4 : #MAX value for columns of the current environment
                Col += 1
                agent.performance -= 1
            else:
                pass
        elif action == 'Left' :
            if agent.location[0] > 0 : #MIN value for columns of the current environment
                Col -= 1
                agent.performance -= 1
            else:
                pass
        agent.location = (Col, Row)


    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B, loc_C, loc_D, loc_E])
    
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_A = Button(self.root, height=4, width=10, padx=2, pady=2, bg='white')
        button_A.config(command=lambda btn=button_A: self.dirt_switch(btn))
        self.buttons.append(button_A)
        button_A_window = self.canvas.create_window(50, 200, anchor=N, window=button_A)

        button_B = Button(self.root, height=4, width=10, padx=2, pady=2, bg='white')
        button_B.config(command=lambda btn=button_B: self.dirt_switch(btn))
        self.buttons.append(button_B)
        button_B_window = self.canvas.create_window(150, 200, anchor=N, window=button_B)

        button_C = Button(self.root, height=4, width=10, padx=2, pady=2, bg='white')
        button_C.config(command=lambda btn=button_C: self.dirt_switch(btn))
        self.buttons.append(button_C)
        button_C_window = self.canvas.create_window(250, 200, anchor=N, window=button_C)
        
        button_D = Button(self.root, height=4, width=10, padx=2, pady=2, bg='white')
        button_D.config(command=lambda btn=button_D: self.dirt_switch(btn))
        self.buttons.append(button_D)
        button_D_window = self.canvas.create_window(350, 200, anchor=N, window=button_D)
        
        button_E = Button(self.root, height=4, width=10, padx=2, pady=2, bg='white')
        button_E.config(command=lambda btn=button_E: self.dirt_switch(btn))
        self.buttons.append(button_E)
        button_E_window = self.canvas.create_window(450, 200, anchor=N, window=button_E)
        
        
        header_button = Button(self.root, text='Vacuum Environment \n AgPos: \n AgPerf: ', 
                               font=('Helvetica', 14), height=12, width=50, padx=2, pady=2 )
        header_button.config(command=lambda btn=header_button: self.display_env(self, self.agents[0]))
        header_button.pack(side='left')
        self.buttons.append(header_button)

    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')
            
    def display_env(self, agent):
        """Updates the GUI according to the agent's action."""
        if self.agents != [] :
            Msg = 'Vacuum Environment \n  AgPos: (' + str(agent.location[0]) + ',' + str(agent.location[1]) + ') \n AgPerf: ' + str(agent.performance)
        else :
            Msg = 'Vacuum Environment \n  AgPos: unk \n AgPerf: 20'
        self.buttons[5].config(text=Msg)

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons[:-1]):
            if btn['bg'] == 'white':
                self.status[(i,0)] = 'Clean'
            else:
                self.status[(i,0)] = 'Dirty'
        #^^ Edit ^^: migliorato

    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        before_step = agent.location
        self.step()   #execute_actions
        move_agent(self, agent, before_step)

def ReflexVacuumAgent():
    """RBAS: A reflex agent for the three-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean', (2,0) : 'Clean'}
    True
    """

    def program(percept):
        NextAction = 'Right'
        AgLocation, status = percept
        MinDist=6
        Cleaned=1
        DirtyRooms=[]
        if status[AgLocation] == 'Dirty' :  #Found the dirty room
                print('Dirty Room found: the agent sucks')
                return 'Suck'
            
        #Search for a dirty room here
        for Room in range(5) :
            if status[(Room,0)]== 'Dirty' :
                Cleaned=0
                #if (Dirty-AgLocation[0]) >= (Dirty - Room):
                DirtyRooms.append(Room)
        
        print("Dirty rooms: ",DirtyRooms)
                
        #Now decide the action
        if Cleaned == 1 :  #there is no dirty room 
            NextAction = 'NoOp'
        else:  #there are Dirty rooms
            for DirtyRoom in DirtyRooms:
                if abs(DirtyRoom - AgLocation[0]) < MinDist: #Edit: l'agente va nella direzione della stanza sporca più vicina.
                    Dirty=DirtyRoom
                    MinDist = DirtyRoom - AgLocation[0]
                    print("la stanza più vicina è la ", Dirty)
            if Dirty > AgLocation[0]  :
                    NextAction = 'Right'
            elif Dirty < AgLocation[0] :
                    NextAction = 'Left'
            else:
                NextAction = 'NoOp'
        print('Selected Action: ', NextAction)
        return NextAction
    return Agent(program)

def create_agent(env, agent):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)
    agent.performance = 0
    env.agent_logo = env.canvas.create_image(10 + (100 * agent.location[0]), 100,anchor=NW, image=MyImage)   
    #^^ Edit ^^ migliorato
    
def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location == before_step:
        pass
    else:
            if agent.location[0]>before_step[0]:
                env.canvas.move(env.agent_logo, 100, 0)
            elif agent.location[0]<before_step[0]:
                env.canvas.move(env.agent_logo,-100, 0)
    # ^^Edit^^: migliorato
    env.display_env(agent)
    
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x575")
    root.resizable(0, 0)
    global MyImage
    MyImage=  PhotoImage(file="AgentVacuum.png")
    frame = Frame(root, bg='black')
    next_button = Button(frame, text='Next', font=('Helvetica', 14), height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent)
    next_button.config(command=lambda: env.update_env(agent))
    root.mainloop()

if __name__ == "__main__":
    main()


Agent has now 
 	 the perception  {(0, 0): 'Dirty', (1, 0): 'Clean', (2, 0): 'Clean', (3, 0): 'Clean', (4, 0): 'Clean'}  
 	 being at Location= (2, 0)  
 	 with performance  0
Dirty rooms:  [0]
la stanza più vicina è la  0
Selected Action:  Left
Agent has now 
 	 the perception  {(0, 0): 'Dirty', (1, 0): 'Clean', (2, 0): 'Clean', (3, 0): 'Clean', (4, 0): 'Dirty'}  
 	 being at Location= (1, 0)  
 	 with performance  -1
Dirty rooms:  [0, 4]
la stanza più vicina è la  0
Selected Action:  Left
Agent has now 
 	 the perception  {(0, 0): 'Clean', (1, 0): 'Clean', (2, 0): 'Clean', (3, 0): 'Clean', (4, 0): 'Dirty'}  
 	 being at Location= (0, 0)  
 	 with performance  -2
Dirty rooms:  [4]
la stanza più vicina è la  4
Selected Action:  Right
